RMinimum : Full - Test - Case: $k(n) = n^{\varepsilon}$ - $n$ fix

In [58]:
import math
import random
import queue

Testfall : $X=[0, \cdots, n-1]$, $k = n^{\varepsilon}$

In [12]:
# User input
logn = 16

# Automatic
n = 2**logn
X = [i for i in range(n)]
lst_k, lst_eps = [], []
can = [1/2, 1/3, 1/4, 1/5, 1/6, 1/7, 1/8, 1/9, 1/10, 1/11, 1/12, 1/16]
for i in range(2, 65):
    ex = 1/i
    k = n**ex
    if k.is_integer() and k > 1:
        lst_k.append(int(k))
        lst_eps.append(round(ex, 2))

# Show Testcase
print('')
print('Input tuples : (n, k), eps')
print('=====================')
for i in range(len(lst_k)):
    print(i+1, ':', '(', int(2**logn), ',', str(lst_k[i]).ljust(4), ') ,', lst_eps[i])


Input tuples : (n, k), eps
1 : ( 65536 , 256  ) , 0.5
2 : ( 65536 , 16   ) , 0.25
3 : ( 65536 , 4    ) , 0.12
4 : ( 65536 , 2    ) , 0.06


Algorithmus : Full

In [60]:
def rminimum(X,k, cnt = []):
    k = int(k)
    n = len(X)
    if cnt == []:
        cnt = [0 for _ in range(len(X))]

    if len(X) == 3:
        if X[0] < X[1]:
            cnt[X[0]] += 2
            cnt[X[1]] += 1
            cnt[X[2]] += 1

            if X[0] < X[2]:
                X = X[0]
            else:
                X = X[2]

        else:
            cnt[X[0]] += 1
            cnt[X[1]] += 2
            cnt[X[2]] += 1

            if X[1] < X[2]:
                X = X[1]
            else:
                X = X[2]

        return cnt

    W, L, cnt = RMinimum_step1(X, cnt)
    minele, cnt = RMinimum_step2(L, k, cnt)
    res3, cnt = RMinimum_step3(W, k, minele, cnt)
    res4, cnt = RMinimum_step4(res3, k, n, cnt)

    return cnt

# ==================================================
def RMinimum_step1(lst, cnt):

    random.shuffle(lst)

    W = [0 for _ in range(len(lst) // 2)]
    L = [0 for _ in range(len(lst) // 2)]

    for i in range(len(lst) // 2):
        if lst[2 * i] > lst[2 * i + 1]:
            W[i] = lst[2 * i + 1]
            L[i] = lst[2 * i]
        else:
            W[i] = lst[2 * i]
            L[i] = lst[2 * i + 1]
        cnt[lst[2 * i + 1]] += 1
        cnt[lst[2 * i]] += 1

    return W, L, cnt

# ==================================================
def RMinimum_step2(L, k, cnt):

    random.shuffle(L)
    res = [L[i * k:(i + 1) * k] for i in range((len(L) + k - 1) // k)]
    minele = [0 for _ in range(len(res))]

    var = list(res)
    for i in range(len(var)):
        q = queue.Queue()

        for item in var[i]:
            q.put(item)

        while q.qsize() > 1:
            a = q.get()
            b = q.get()

            if a < b:
                q.put(a)
            else:
                q.put(b)
            cnt[a] += 1
            cnt[b] += 1
        minele[i] = q.get()

    return minele, cnt

# ==================================================
def RMinimum_step3(lst, k, minele, cnt):

    random.shuffle(lst)
    var = [lst[i * k:(i + 1) * k] for i in range((len(lst) + k - 1) // k)]
    res = [0 for _ in range(len(var))]

    for i in range(len(var)):
        res[i] = [elem for elem in var[i] if elem < minele[i]]
        cnt[minele[i]] += len(var[i])
        for elem in var[i]:
            cnt[elem] += 1

    res = [item for sublist in res for item in sublist]

    return res, cnt

# ==================================================
def RMinimum_step4(newW, k, n, cnt):

    if len(newW) <= (math.log(n)/math.log(2))**2:
        q = queue.Queue()

        var = list(newW)
        for item in var:
            q.put(item)
        while q.qsize() > 1:
            a = q.get()
            b = q.get()

            if a < b:
                q.put(a)
            else:
                q.put(b)

            cnt[a] += 1
            cnt[b] += 1
        res = q.get()

    else:
        res = rminimum(newW,k, cnt)

    return res, cnt

# ==================================================
# Testfall
lst_cnt = []
for k in lst_k:
    lst_cnt.append(rminimum(X, k))

Resultat :

In [72]:
def test(n, lst_k, lst_eps, lst_cnt):
    
    print('')
    print('E[f_min] = O(eps^(-1) * loglog(n))   |   E[f_rem] = O(n^eps)   |   E[work] = O(n)')

    for i in range(len(lst_k)):
        f_min = lst_cnt[i][0]
        f_rem = max(lst_cnt[i][1:])
        f_avg = round(sum(lst_cnt[i]) / len(lst_cnt[i]), 2)
        fg = max(lst_cnt[i])
        work = int(sum(lst_cnt[i])/2)
        E_min = round((lst_eps[i]**(-1) * (math.log(n)/math.log(2)) / (math.log(math.log(n)/math.log(2))/math.log(2))), 2)
        E_rem = round(n**lst_eps[i], 2)
        E_work = n
        diff_min = round(abs(f_min - E_min), 2)
        diff_rem = round(abs(f_rem - E_rem), 2)
        diff_work = round(abs(work - E_work), 2)
        
        test_1_work = (2 - lst_eps[i])*n
        test_1_diff = round(abs(work - test_1_work), 2)
        
        print('')
        print('n :', n, '        k :', int(lst_k[i]), '        eps :', lst_eps[i])
        print('================================================')
        print('f_min :', f_min, '   E[f_min] :', E_min, '   Diff :', diff_min)
        print('------------------------------------------------')    
        print('f_rem :', f_rem, '   E[f_rem] :', E_rem, '      Diff :', diff_rem)
        print('------------------------------------------------')
        print('Work  :', work, '   E[work] :', E_work, '   Diff :', diff_work)
        print('------------------------------------------------')
        print('Fragile Complexity :', fg, '   Avg(fg) :', f_avg)
        print('================================================')
        print('Test 1: E[work] ~ (2 - eps) * n')
        print('Work  :', work, '   E[work] :', test_1_work, '   Diff :', test_1_diff)
        print('================================================')
        print('')
    return

# ==================================================
# Testfall
test(n, lst_k, lst_eps, lst_cnt)


E[f_min] = O(eps^(-1) * loglog(n))   |   E[f_rem] = O(n^eps)   |   E[work] = O(n)

n : 65536         k : 256         eps : 0.5
f_min : 10    E[f_min] : 8.0    Diff : 2.0
------------------------------------------------
f_rem : 267    E[f_rem] : 256.0       Diff : 11.0
------------------------------------------------
Work  : 104187    E[work] : 65536    Diff : 38651
------------------------------------------------
Fragile Complexity : 267    Avg(fg) : 3.18
Test 1: E[work] ~ (2 - eps) * n
Work  : 104187    E[work] : 98304.0    Diff : 5883.0


n : 65536         k : 16         eps : 0.25
f_min : 15    E[f_min] : 16.0    Diff : 1.0
------------------------------------------------
f_rem : 29    E[f_rem] : 16.0       Diff : 13.0
------------------------------------------------
Work  : 118486    E[work] : 65536    Diff : 52950
------------------------------------------------
Fragile Complexity : 29    Avg(fg) : 3.62
Test 1: E[work] ~ (2 - eps) * n
Work  : 118486    E[work] : 114688.0    Diff 

In [64]:
a = [0,1,2, 44]
b = sum(a) / len(a)
print(b)

11.75
